In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time                   # To time processes
import warnings               # To suppress warnings
import itertools
import datetime
import csv

from random import shuffle
from pandas import read_csv
from sklearn import metrics
from sklearn import svm
from matplotlib import pyplot
from numpy import array
from numpy import argmax
from scipy import stats

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans

from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.base import TransformerMixin
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import RandomOverSampler

from matplotlib.pyplot import *

warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
dataset = read_csv("cs_mitx.csv", encoding="ISO-8859-1")

In [3]:
df=dataset[['viewed','explored', 'grade', 'nevents', 'ndays_act', 'nchapters','LoE_DI','last_event_DI']]

idx = pd.date_range('01-01-2012', '12-30-2013')
df['last_event_DI'] = pd.to_datetime(df['last_event_DI'])
print(df['last_event_DI'].head())

0   2012-12-21
1          NaT
2   2012-10-18
3   2012-04-11
4   2012-09-26
Name: last_event_DI, dtype: datetime64[ns]


In [4]:
#Impute missing dates
j=0
for ix in df.index:
    #print(df.loc[ix]['last_event_DI'])
    if(str(df.loc[ix]['last_event_DI']) == 'NaT'):
        #print(ix,"\n")
        #print(idx[j%30])
        df.at[ix, 'last_event_DI'] = idx[j%730]
        j=j+1

print(df['last_event_DI'].head())

0   2012-12-21
1   2012-01-01
2   2012-10-18
3   2012-04-11
4   2012-09-26
Name: last_event_DI, dtype: datetime64[ns]


In [5]:
#Converting Last event date into unix timestamp
df['last_event_DI'] = df['last_event_DI'].astype(str)
end_temp= np.zeros(shape=(df['last_event_DI'].shape[0],1))
for ix in df['last_event_DI'].index:
    end_temp[ix]=int(time.mktime(datetime.datetime.strptime(df.loc[ix]['last_event_DI'], "%Y-%m-%d").timetuple()))

df_temp2=pd.DataFrame({'Last event date':end_temp[:,0]})
df_temp2.head()

df=pd.concat([df,df_temp2], axis=1)

In [6]:
#One-hot encoding for converting Level of Education
data = df[['LoE_DI']]
values = array(data)
#print(values)
values.ravel()
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape((integer_encoded.shape[0], 1))
df[['LoE_DI']]=integer_encoded

In [7]:
# for normalizing grades and gender missing values

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


df[['grade']] = DataFrameImputer().fit_transform(df[['grade']])

In [8]:
df.isnull().sum()

viewed             0
explored           0
grade              0
nevents            0
ndays_act          0
nchapters          0
LoE_DI             0
last_event_DI      0
Last event date    0
dtype: int64

In [9]:
df.head()

,viewed,explored,grade,nevents,ndays_act,nchapters,LoE_DI,last_event_DI,Last event date
0,1,0,0.00,43,1,3,0,2012-12-21,1.356028e+09
1,0,0,0.00,0,0,0,0,2012-01-01,1.325356e+09
2,1,0,0.01,879,5,3,4,2012-10-18,1.350499e+09
3,1,0,0.00,547,8,2,0,2012-04-11,1.334083e+09
4,0,0,0.00,2,1,0,4,2012-09-26,1.348598e+09


In [10]:
df=df.drop(['last_event_DI'], axis=1)

In [11]:
#Clustering for certified or not

df11=df[['viewed','explored','grade','ndays_act','nchapters','LoE_DI']]
print(type(df11))

kmeans = KMeans(n_clusters=2)
kmeans.fit(df11)
y_km = kmeans.fit_predict(df11)

df7= pd.DataFrame({'certified':y_km})
df7=df7.values

print("Week ",j+1,"\n",np.array(np.unique(y_km, return_counts=True)).T)
df=np.concatenate((df,df7),axis=1)

<class 'pandas.core.frame.DataFrame'>
Week  4105 
 [[    0 56931]
 [    1  2348]]


In [12]:
type(df)

numpy.ndarray

In [13]:
print(df[0])

[1.0000000e+00 0.0000000e+00 0.0000000e+00 4.3000000e+01 1.0000000e+00
 3.0000000e+00 0.0000000e+00 1.3560282e+09 0.0000000e+00]


In [14]:
#Creating file for test dataset from cs_mitx
np.savetxt("Testing dataset.csv", df, delimiter=",")

df5 = read_csv('Testing dataset.csv')
df5.columns=['viewed','explored', 'grade', 'nevents', 'ndays_act', 'nchapters','LoE_DI','last_event_DI', 'certified']
df5.to_csv('Testing dataset.csv')
df5 = df5.drop(df5.index, inplace=True)

In [15]:
print(dataset['Completed_or_Not'].value_counts())

0    57539
1     1740
Name: Completed_or_Not, dtype: int64


In [16]:
df77=dataset['Completed_or_Not']
df77.to_csv('Testing result.csv')

In [17]:
df88=read_csv('Testing result.csv')
print(df88['Completed_or_not'].value_counts())

0    57538
1     1740
Name: Completed_or_not, dtype: int64
